In [31]:
import os
os.environ['PROJ_LIB'] = r'C:\Users\osori050\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Library\share\proj'

In [32]:
import io
import zipfile
import requests
import geopandas as gpd
import pandas as pd
from itertools import product
import numpy as np
import math
import shutil
from scipy.stats import zscore
import fiona
from fiona.crs import from_epsg
from shapely import geometry
import psycopg2

In [33]:
# Set workspace
os.chdir(r'E:\ArcGIS_2\Lab4')
wksp = os.getcwd()

In [135]:
# Project cities back to UTM 15N
sr = arcpy.SpatialReference(26915)
arcpy.Project_management('cities.shp', 'cities_projected.shp', sr)

<Result 'E:\\ArcGIS_2\\Lab4\\cities_projected.shp'>

In [133]:
# Cities with stink bug presence data frame
affected_cities = gpd.read_file('sbug_cities.shp')
affected_cities.drop(affected_cities.iloc[:, np.r_[0, 2:10]], axis=1, inplace=True) 
affected_cities['Presence'] = 1

In [136]:
# All cities data frame
cities_gdf = gpd.read_file('cities_projected.shp')
cities_gdf.drop(cities_gdf.iloc[:, np.r_[0, 2:6, 7:9]], axis=1, inplace=True) 

# Add presence to cities data frame
cities_gdf = cities_gdf.merge(affected_cities, on='FEATURE_NA', how='left')
cities_gdf['Presence'] = cities_gdf['Presence'].fillna(0)

In [137]:
# Read mean min temp shapefile
temp = gpd.read_file("cities_min_temp.shp")
temp.drop(temp.iloc[:, np.r_[0, 2:9, 10]], axis=1, inplace=True)
temp.reset_index(inplace=True)

# Join data to cities
cities_gdf.reset_index(inplace=True)
cities_gdf = cities_gdf.merge(temp, on='index', how='inner')
cities_gdf.drop(cities_gdf.iloc[:, np.r_[0, 5]], axis=1, inplace=True)

In [138]:
# Calculate z-values for population and temp
cols = ['POPULATION', 'min_temp']
z_val = cities_gdf[cols].apply(zscore)
cities_gdf = cities_gdf.join(z_val, rsuffix=" (Z)")

# Move the distribution so the starting poing is zero
cities_gdf["POPULATION (Z)"] = cities_gdf["POPULATION (Z)"].apply(lambda x: abs(cities_gdf["POPULATION (Z)"].min()) + x)
cities_gdf["min_temp (Z)"] = cities_gdf["min_temp (Z)"].apply(lambda x: abs(cities_gdf["min_temp (Z)"].min()) + x)

# Only cities with presencce
cities_with_presence = cities_gdf[cities_gdf["Presence"] == 1].copy()

In [139]:
# Combine each city with the others in MN except with itself
combinations = product(cities_with_presence.values, cities_gdf.values)
df_combined = pd.DataFrame([*combinations], columns=['From', 'To'])
df_combined[['From', 'To']] = df_combined[['From', 'To']].applymap(lambda x: x[0])
df_combined = df_combined.loc[df_combined['From'] != df_combined['To']]
df_combined.reset_index(drop=True, inplace=True)

In [140]:
# Add geometry to 'From' and 'To' cities
df_combined = df_combined.merge(cities_gdf, left_on='From', right_on='FEATURE_NA_x', how='inner')
df_combined = df_combined.merge(cities_gdf, left_on='To', right_on='FEATURE_NA_x', how='left')
df_combined.drop(df_combined.iloc[:, np.r_[2, 3, 6, 9, 10, 13]], axis=1, inplace=True)
df_combined.drop_duplicates(inplace=True) # Remove duplicates generated in the previous join
df_combined.reset_index(drop=True, inplace=True)

In [142]:
# Calculate weight
df_combined['W From'] = df_combined['POPULATION (Z)_x'] + df_combined['min_temp (Z)_x']
df_combined['W To'] = df_combined['POPULATION (Z)_y'] + df_combined['min_temp (Z)_y']

In [145]:
# Calculate centroids
df_combined['centroid_from'] = ''
df_combined['centroid_to'] = ''
for i in range(len(df_combined)):
    df_combined.at[i, 'centroid_from'] = df_combined['geometry_x'][i].centroid.coords[0]
    df_combined.at[i, 'centroid_to'] = df_combined['geometry_y'][i].centroid.coords[0]
    
# Calculate euclidean distance
df_combined['Distance'] = ''
for i in range(len(df_combined)):
    df_combined.at[i, 'Distance'] = math.dist(df_combined.at[i, 'centroid_from'], df_combined.at[i, 'centroid_to']) / 1000 # km

In [149]:
# Remove unneeded columns
df_combined.drop(df_combined.iloc[:, np.r_[2, 4:7, 8, 9, 12, 13]], axis=1, inplace=True)

# Rename columns
df_combined = df_combined.rename(columns={'Presence_x': 'Presence From', 
                                            'Presence_y': 'Presence To'})

In [151]:
# Build the 3 different models
gravity = df_combined.copy()
gravity['Force'] = (gravity['W From']*gravity['W To'])/gravity['Distance']

huff_simple = df_combined.copy()
huff_simple['Force'] = huff_simple['W To']/huff_simple['Distance']

huff_faster_decay = df_combined.copy()
huff_faster_decay['Force'] = huff_faster_decay['W To']/(huff_faster_decay['Distance']**2)

In [161]:
def modeling(model):
    # Calculate total force for each from-city
    force_per_from_city = model.groupby('From').agg({'Force': 'sum'}).reset_index()
    force_per_from_city = force_per_from_city.rename(columns={'Force': 'Total force per from-city'})
    model = model.merge(force_per_from_city, on='From', how='left')

    # Probability of transition
    model['Prob_trans'] = 0
    for i in range(len(model)):
        if model.at[i, 'Total force per from-city'] != 0:
            model.at[i, 'Prob_trans'] = model.at[i, 'Force']/model.at[i, 'Total force per from-city']
        else:
            model.at[i, 'Total force per from-city'] = 0
            
    # Monte carlo simulation
    for i in range (0,100):
        # Get draw from flat distribution
        transition_draw = np.random.random()

        # Predict affected cities
        for index, row in model.iterrows():
            if row["Prob_trans"] > transition_draw:
                model.loc[index, "Presence To"] = 1
                
    # Calculate risk of invasion for each city
    risk = model.groupby('To').agg({'Prob_trans': 'sum'}).reset_index()
    risk = risk.rename(columns={'Prob_trans': 'Risk'})
    
    # Extract potential cities affected into a new data frame
    predicted = model[model["Presence To"] == 1].copy()
    predicted = predicted.groupby('To').agg({'Presence To': 'min'}).reset_index()
    
    # Read city shapefile again and join the prediction and risk
    cities_gdf = gpd.read_file('cities_projected.shp')
    cities_gdf.drop(cities_gdf.iloc[:, np.r_[0, 2:9]], axis=1, inplace=True) 
    cities_gdf = cities_gdf.rename(columns={'FEATURE_NA': 'Name'})
    cities_gdf = cities_gdf.merge(predicted, left_on='Name', right_on='To', how='left')
    cities_gdf = cities_gdf.merge(risk, left_on='Name', right_on='To', how='left')
    cities_gdf.drop(cities_gdf.iloc[:, np.r_[2, 4]], axis=1, inplace=True) 
    cities_gdf = cities_gdf.rename(columns={'Presence To': 'Presence'})
    cities_gdf['Presence'] = cities_gdf['Presence'].fillna(0)
    
    return cities_gdf

In [162]:
# Simulate the models and create geo data frames with the outputs
gravity_gdf = modeling(gravity)

huff_simple_gdf = modeling(huff_simple)

huff_faster_decay_gdf = modeling(huff_faster_decay)

In [163]:
gravity_results = gravity_gdf[gravity_gdf["Presence"] == 1].copy()
print(f"Number of cities predicted with the gravity model: {len(gravity_results['Name'].unique())}")

simple_results = huff_simple_gdf[huff_simple_gdf["Presence"] == 1].copy()
print(f"Number of cities predicted with the huff simple model: {len(simple_results['Name'].unique())}")

faster_decay_results = huff_faster_decay_gdf[huff_faster_decay_gdf["Presence"] == 1].copy()
print(f"Number of cities predicted with the huff model when alpha=2: {len(faster_decay_results['Name'].unique())}")

Number of cities predicted with the gravity model: 62
Number of cities predicted with the huff simple model: 467
Number of cities predicted with the huff model when alpha=2: 160


In [160]:
huff_faster_decay_gdf

,Name,geometry,Presence,Risk
0,Bluffton,"POLYGON ((-95.21819 46.45608, -95.23910 46.456...",0.0,0.000872
1,Sartell,"MULTIPOLYGON (((-94.26356 45.60481, -94.25788 ...",1.0,0.108466
2,Cambridge,"MULTIPOLYGON (((-93.26423 45.57300, -93.25328 ...",0.0,0.015013
3,Waseca,"MULTIPOLYGON (((-93.54707 44.07307, -93.54201 ...",0.0,0.007439
4,La Crescent,"MULTIPOLYGON (((-91.33659 43.82339, -91.33646 ...",0.0,0.005428
...,...,...,...,...
898,Plainview,"POLYGON ((-92.18964 44.17411, -92.17971 44.173...",0.0,0.014406
899,Albertville,"POLYGON ((-93.63381 45.23803, -93.63405 45.224...",0.0,0.017126
900,Storden,"POLYGON ((-95.31084 44.04338, -95.31072 44.036...",0.0,0.001336
901,Rosemount,"POLYGON ((-93.10614 44.77570, -93.09563 44.775...",1.0,0.145193


#### Huff alpha=2 for now

In [164]:
# Define the output shapefile schema
schema = {
    'geometry': 'Polygon',
    'properties': {
        'Name': 'str',
        'Presence': 'int',
        'Risk': 'float'
    }
}

# Set the CRS of the GeoDataFrame
crs = from_epsg(26915)

# Open the output shapefile and write the GeoDataFrame to it
with fiona.open('cities_results.shp', 'w', driver='ESRI Shapefile', schema=schema, crs=crs) as output:
    for index, row in huff_faster_decay_gdf.iterrows():
        output.write({
            'geometry': geometry.mapping(row.geometry),
            'properties': {
                'Name': row['Name'],
                'Presence': row['Presence'],
                'Risk': row['Risk']
            }
        })

In [165]:
# Re-project cities to WGS 1984
sr = arcpy.SpatialReference(4326)
arcpy.Project_management('cities_results.shp', 'cities_results_projected.shp', sr)

<Result 'E:\\ArcGIS_2\\Lab4\\cities_results_projected.shp'>

In [166]:
# Connect to postgresql database
connection = psycopg2.connect(host = '34.27.219.64',
                              port = '5432',
                              database = 'lab1',
                              user = 'postgres',
                              password = 'student',
                             )

In [167]:
data = ("cities_results_projected.shp")
fields = ["Name", "Presence", "Risk", "Shape@WKT"]

# Create SQL table
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS cities")
cursor.execute("""
    CREATE TABLE cities (
        id SERIAL,
        Name VARCHAR,
        Presence INTEGER,
        Risk DOUBLE PRECISION)
""")

cursor.execute("""
    SELECT AddGeometryColumn('cities', 'geom', 4326, 'MULTIPOLYGON', 2)
""")

# Populate PostGIS
with arcpy.da.SearchCursor(data, fields) as da_cursor:
    for row in da_cursor:
        wkt = row[3]
        cursor.execute("INSERT INTO cities (Name, Presence, Risk, geom) VALUES (%s, %s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], row[2], wkt))

connection.commit()
connection.close()